In [1]:
import pandas as pd
import pyproj
import xarray as xr

# Load data from CSV
df = pd.read_csv('Windraeder.csv', encoding='ISO-8859-1')

# Filter operational turbines
df = df[df['operationalStatus'] == 'operationalStatus_normalbetrieb']

# Swiss coordinate system to WGS84 converter
def swiss_to_wgs84(x, y):
    swiss = pyproj.CRS('EPSG:2056')
    wgs84 = pyproj.CRS('EPSG:4326')
    transformer = pyproj.Transformer.from_crs(swiss, wgs84, always_xy=True)
    lon, lat = transformer.transform(x, y)
    return lon, lat

# Convert coordinates
df[['longitude', 'latitude']] = df.apply(lambda row: swiss_to_wgs84(row['x'], row['y']), axis=1, result_type='expand')

# Select required columns and convert to xarray
selected_columns = ['xtf_id', 'hubHeight', 'diameter', 'ratedPower', 'latitude', 'longitude']
df_selected = df[selected_columns]
xarray_data = df_selected.to_xarray()

# Convert xarray DataArray to pandas DataFrame
df = xarray_data.to_dataframe()

# Set pandas display options (optional)
#pd.set_option('display.max_rows', None)  # for unlimited rows
#pd.set_option('display.max_columns', None)  # for unlimited columns

# Print the DataFrame
print(df)
print(xarray_data)

           xtf_id  hubHeight  diameter  ratedPower   latitude  longitude
index                                                                   
0       turbine_5        100        71        2000  46.159843   7.037331
1       turbine_8         61        52         900  46.990003   8.086715
2       turbine_9         46        40         600  46.654224   8.616332
8      turbine_31         78        82        2000  47.302711   7.100815
9      turbine_32         99        82        2000  46.126610   7.052194
10     turbine_33         55        44         900  46.654893   8.612466
11     turbine_34         95        90        2000  47.176891   7.015705
12     turbine_35        108        82        2300  47.199788   6.955356
13     turbine_38         85        71        2310  46.463039   8.374565
14     turbine_39         50        54         950  46.987077   8.087087
15     turbine_41         99       101        3000  46.131138   7.145424
16     turbine_42         55        44         900 

In [5]:
import sqlite3
import pandas as pd
import xarray as xr

# Connexion à la base de données SQLite
conn = sqlite3.connect('data/data.sqlite')
cursor = conn.cursor()

# Exécuter une requête SQL
cursor.execute("SELECT EGID, GKODE, GKODN, GKAT FROM building WHERE GKAT<1050")
rows = cursor.fetchall()

# Fermer la connexion
conn.close()

# Stocker les résultats dans un tableau (liste)
data = list(rows)

def convert_coords(gkode, gkodn):
    # Check if the coordinates are empty strings
    if gkode == '' or gkodn == '':
        # Handle the empty string case (e.g., return None or a default value)
        return None, None
    else:
        # Convert string coordinates to float
        gkode_float = float(gkode)
        gkodn_float = float(gkodn)
        
        # Perform the transformation
        lon, lat = pyproj.transformer.transform(gkode_float, gkodn_float)
        return lat, lon

# Convert and store the coordinates
converted_data = []
for row in data:
    egid, gkode, gkodn, gkat = row
    lat, lon = convert_coords(gkode, gkodn)
    
    # Skip rows with missing coordinates
    if lat is not None and lon is not None:
        converted_data.append((egid, lat, lon, gkat))


# Assuming converted_data is structured as [(egid, lat, lon, gkat), ...]

# Convert converted_data to a pandas DataFrame
df = pd.DataFrame(converted_data, columns=['EGID', 'Latitude', 'Longitude', 'GKAT'])

# Convert the DataFrame to an xarray DataArray
houses_array = xr.DataArray(df, dims=["building", "attribute"])

# Print the xarray DataArray
print(houses_array)

TypeError: transform() missing 2 required positional arguments: 'x' and 'y'

In [ ]:
import numpy as np
from haversine import haversine
import sqlite3
import pandas as pd
import xarray as xr

import pandas as pd
import pyproj

# Load data from CSV
df = pd.read_csv('Windraeder.csv', encoding='ISO-8859-1')

# Filter operational turbines
df = df[df['operationalStatus'] == 'operationalStatus_normalbetrieb']

# Define the transformer for coordinate conversion
swiss = pyproj.CRS('EPSG:2056')  # Swiss coordinate system (LV95)
wgs84 = pyproj.CRS('EPSG:4326')  # WGS84 coordinate system
transformer = pyproj.Transformer.from_crs(swiss, wgs84, always_xy=True)


# Swiss coordinate system to WGS84 converter
def swiss_to_wgs84(x, y):
    swiss = pyproj.CRS('EPSG:2056')
    wgs84 = pyproj.CRS('EPSG:4326')
    transformer = pyproj.Transformer.from_crs(swiss, wgs84, always_xy=True)
    lon, lat = transformer.transform(x, y)
    return lon, lat

def convert_coords(gkode, gkodn):
    # Check if the coordinates are empty strings
    if gkode == '' or gkodn == '':
        # Handle the empty string case (e.g., return None or a default value)
        return None, None
    else:
        # Convert string coordinates to float
        gkode_float = float(gkode)
        gkodn_float = float(gkodn)
        
        # Perform the transformation
        lon, lat = transformer.transform(gkode_float, gkodn_float)
        return lat, lon
    
# Vectorized haversine function
def haversine_vectorized(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Multiply by Earth's radius to get kilometers
    return km    
    

# Convert coordinates
df[['longitude', 'latitude']] = df.apply(lambda row: swiss_to_wgs84(row['x'], row['y']), axis=1, result_type='expand')

# Select required columns and convert to xarray
selected_columns = ['xtf_id', 'hubHeight', 'diameter', 'ratedPower', 'latitude', 'longitude']
df_selected = df[selected_columns]
xarray_data = df_selected.to_xarray()
# Convert xarray DataArray to pandas DataFrame
df = xarray_data.to_dataframe()

# Set pandas display options (optional)
#pd.set_option('display.max_rows', None)  # for unlimited rows
#pd.set_option('display.max_columns', None)  # for unlimited columns

# Print the DataFrame
#print(df)
#print(xarray_data)
#------------------------------------------------------------------------------------------------------------------------------------------------


# Connexion à la base de données SQLite
conn = sqlite3.connect('data/data.sqlite')
cursor = conn.cursor()

# Exécuter une requête SQL
cursor.execute("SELECT EGID, GKODE, GKODN, GKAT FROM building WHERE GKAT<1050")
rows = cursor.fetchall()

# Fermer la connexion
conn.close()

# Stocker les résultats dans un tableau (liste)
data = list(rows)

# Convert and store the coordinates
converted_data = []
for row in data:
    egid, gkode, gkodn, gkat = row
    lat, lon = convert_coords(gkode, gkodn)
    
    # Skip rows with missing coordinates
    if lat is not None and lon is not None:
        converted_data.append((egid, lat, lon, gkat))


# Assuming converted_data is structured as [(egid, lat, lon, gkat), ...]

# Convert converted_data to a pandas DataFrame
dfhouses = pd.DataFrame(converted_data, columns=['EGID', 'Latitude', 'Longitude', 'GKAT'])

# Convert the DataFrame to an xarray DataArray
houses_array = xr.DataArray(dfhouses, dims=["building", "attribute"])

# Print the xarray DataArray
print(houses_array)

# Extract coordinates from the xarray DataArrays
# Correct the order of coordinates (swap latitude and longitude)
buildings_coords = np.array([houses_array.sel(attribute='Latitude').values,
                             houses_array.sel(attribute='Longitude').values]).T
turbines_coords = np.array([xarray_data.latitude.values,
                            xarray_data.longitude.values]).T

# Ensure all coordinates are of type float
buildings_coords = buildings_coords.astype(float)
turbines_coords = turbines_coords.astype(float)

# Perform the broadcasting and distance calculation
distances = haversine_vectorized(buildings_coords[:, np.newaxis, 0], buildings_coords[:, np.newaxis, 1],
                                 turbines_coords[np.newaxis, :, 0], turbines_coords[np.newaxis, :, 1])

# Find the minimum distance for each building
min_distances = distances.min(axis=1)

# Filter buildings where the closest turbine is within 1km
close_buildings_mask = min_distances <= 1  # 1km
filtered_buildings = buildings_coords[close_buildings_mask]

# Result
print(filtered_buildings)

# Initialize a list to store the results
results = []

# Iterate through each building
for i, building_coord in enumerate(buildings_coords):
    # Calculate distances to all turbines
    distances_to_turbines = haversine_vectorized(building_coord[0], building_coord[1], turbines_coords[:, 0], turbines_coords[:, 1])
    
    # Filter turbines within 1 km
    turbines_within_1km = np.where(distances_to_turbines <= 1)[0]
    
    # If there are turbines within 1 km, store the building's coordinates and the distances
    for turbine_index in turbines_within_1km:
        results.append((building_coord[0], building_coord[1], distances_to_turbines[turbine_index]))

# Convert the list to a structured numpy array
dtype = [('Latitude', float), ('Longitude', float), ('Distance to Turbine (km)', float)]
numpy_results = np.array(results, dtype=dtype)

# Display the numpy array
print(numpy_results)

In [4]:
# Count the number of buildings within 1 km of a wind turbine
num_buildings_within_1km = len(filtered_buildings)

# Print summary for the user
print(f"Number of buildings within 1 km of a wind turbine: {num_buildings_within_1km}")

Number of buildings within 1 km of a wind turbine: 796


In [32]:
import numpy as np
from haversine import haversine

# Extract coordinates from the xarray DataArrays
buildings_coords = np.array([houses_array.sel(attribute='Latitude').values,
                             houses_array.sel(attribute='Longitude').values]).T
turbines_coords = np.array([xarray_data.latitude.values,
                            xarray_data.longitude.values]).T

# Vectorized haversine function
def haversine_vectorized(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Multiply by Earth's radius to get kilometers
    return km

# Ensure all coordinates are of type float
buildings_coords = buildings_coords.astype(float)
turbines_coords = turbines_coords.astype(float)

# Perform the broadcasting and distance calculation
distances = haversine_vectorized(buildings_coords[:, np.newaxis, 0], buildings_coords[:, np.newaxis, 1],
                                 turbines_coords[np.newaxis, :, 0], turbines_coords[np.newaxis, :, 1])

# Find the minimum distance for each building
min_distances = distances.min(axis=1)

# Filter buildings where the closest turbine is within 1km
close_buildings_mask = min_distances <= 100  # 1km
filtered_buildings = buildings_coords[close_buildings_mask]

# Result
print(filtered_buildings)


[]


In [43]:
buildings_coords = np.array([houses_array.sel(attribute='Longitude').values,
                             houses_array.sel(attribute='Latitude').values]).T

In [44]:
print(buildings_coords)

[[47.283591743499585 8.491434565726783]
 [47.27839821803604 8.504399817374951]
 [47.268628302661675 8.48280806951358]
 ...
 [47.48445445211638 7.105353732987037]
 [47.48457458583299 7.104825265586205]
 [47.4839145918797 7.09674341506249]]


In [45]:
turbines_coords = np.array([xarray_data.latitude.values,
                            xarray_data.longitude.values]).T

In [46]:
print(turbines_coords)

[[51.83605575 36.49984552]
 [52.33732782 38.06734427]
 [51.89703765 38.41141118]
 [52.86972224 37.24298205]
 [51.80172093 36.49594314]
 [51.89861656 38.40784859]
 [52.77576734 37.07927793]
 [52.811797   37.03095096]
 [51.78151038 38.05232068]
 [52.33455194 38.06598543]
 [51.7826137  36.59489946]
 [51.89858342 38.41035789]
 [51.88760199 39.49248294]
 [52.33633558 38.09207837]
 [52.78386281 37.15424114]
 [52.77013136 37.04010197]
 [52.78259289 37.09194497]
 [52.77705124 37.07237629]
 [52.78329838 37.09550042]
 [52.78205204 37.0769091 ]
 [52.7827716  37.08204551]
 [52.7784424  37.08588685]
 [52.78482288 37.1268082 ]
 [52.7740639  37.06339694]
 [52.77962871 37.07783682]
 [52.81187255 37.04004412]
 [52.81238815 37.05028828]
 [51.89878771 38.41370839]
 [52.86899482 37.25004035]
 [51.78014932 38.05460078]
 [51.77982312 38.0564291 ]
 [51.78317488 38.05013082]
 [52.77200658 37.02134143]
 [52.77226187 37.0436822 ]
 [52.78482106 37.1374697 ]
 [52.78367838 37.14201101]
 [51.82899332 38.29664117]
 

In [8]:
import numpy as np

# Function to calculate distances using the haversine formula
def haversine_vectorized(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km

# Initialize a list to store the results
buildings_with_turbine_distances = []

# Iterate through each building
for building_coord in buildings_coords:
    # Calculate distances to all turbines
    distances_to_turbines = haversine_vectorized(building_coord[0], building_coord[1], turbines_coords[:, 0], turbines_coords[:, 1])
    
    # Filter turbines within 1 km and store their distances
    turbines_within_1km = distances_to_turbines[distances_to_turbines <= 1]
    
    # If there are turbines within 1 km, store the building's coordinates and the distances
    if len(turbines_within_1km) > 0:
        buildings_with_turbine_distances.append((building_coord, turbines_within_1km))

# Convert the results to a more structured format if needed
# For example, you can create a DataFrame with columns for building coordinates and turbine distances
df_results = pd.DataFrame(buildings_with_turbine_distances, columns=['Building Coordinates', 'Turbine Distances'])

# Print or process the DataFrame as needed
print(df_results)


Empty DataFrame
Columns: [Building Coordinates, Turbine Distances]
Index: []


In [5]:
import numpy as np
from haversine import haversine
import sqlite3
import pandas as pd
import xarray as xr

import pandas as pd
import pyproj
import xarray as xr

# Load data from CSV
df = pd.read_csv('Windraeder.csv', encoding='ISO-8859-1')
swiss = pyproj.CRS('EPSG:2056')
wgs84 = pyproj.CRS('EPSG:4326')

# Filter operational turbines
df = df[df['operationalStatus'] == 'operationalStatus_normalbetrieb']
transformer = pyproj.Transformer.from_crs(swiss, wgs84, always_xy=True)

# Swiss coordinate system to WGS84 converter
def swiss_to_wgs84(x, y):
    swiss = pyproj.CRS('EPSG:2056')
    wgs84 = pyproj.CRS('EPSG:4326')
    lon, lat = transformer.transform(x, y)
    return lon, lat

# Convert coordinates
df[['longitude', 'latitude']] = df.apply(lambda row: swiss_to_wgs84(row['x'], row['y']), axis=1, result_type='expand')

# Select required columns and convert to xarray
selected_columns = ['xtf_id', 'hubHeight', 'diameter', 'ratedPower', 'longitude', 'latitude']
df_selected = df[selected_columns]
xarray_data = df_selected.to_xarray()

# Convert xarray DataArray to pandas DataFrame
df = xarray_data.to_dataframe()

# Set pandas display options (optional)
#pd.set_option('display.max_rows', None)  # for unlimited rows
#pd.set_option('display.max_columns', None)  # for unlimited columns

# Print the DataFrame
print(df)
print(xarray_data)

# Connexion à la base de données SQLite
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

# Exécuter une requête SQL
cursor.execute("SELECT EGID, GKODE, GKODN, GKAT FROM building WHERE GKAT<1050")
rows = cursor.fetchall()

# Fermer la connexion
conn.close()

# Stocker les résultats dans un tableau (liste)
data = list(rows)

def convert_coords(gkode, gkodn):
    # Check if the coordinates are empty strings
    if gkode == '' or gkodn == '':
        # Handle the empty string case (e.g., return None or a default value)
        return None, None
    else:
        # Convert string coordinates to float
        gkode_float = float(gkode)
        gkodn_float = float(gkodn)
        
        # Perform the transformation
        lon, lat = transformer.transform(gkode_float, gkodn_float)
        return lat, lon

# Convert and store the coordinates
converted_data = []
for row in data:
    egid, gkode, gkodn, gkat = row
    lat, lon = convert_coords(gkode, gkodn)
    
    # Skip rows with missing coordinates
    if lat is not None and lon is not None:
        converted_data.append((egid, lat, lon, gkat))


# Assuming converted_data is structured as [(egid, lat, lon, gkat), ...]

# Convert converted_data to a pandas DataFrame
df = pd.DataFrame(converted_data, columns=['EGID', 'Latitude', 'Longitude', 'GKAT'])

# Convert the DataFrame to an xarray DataArray
houses_array = xr.DataArray(df, dims=["building", "attribute"])

# Print the xarray DataArray
print(houses_array)

# Extract coordinates from the xarray DataArrays
# Correct the order of coordinates (swap latitude and longitude)
buildings_coords = np.array([houses_array.sel(attribute='Longitude').values,
                             houses_array.sel(attribute='Latitude').values]).T
turbines_coords = np.array([xarray_data.latitude.values,
                            xarray_data.longitude.values]).T

# Vectorized haversine function
def haversine_vectorized(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Multiply by Earth's radius to get kilometers
    return km

# Ensure all coordinates are of type float
buildings_coords = buildings_coords.astype(float)
turbines_coords = turbines_coords.astype(float)

# Perform the broadcasting and distance calculation
distances = haversine_vectorized(buildings_coords[:, np.newaxis, 0], buildings_coords[:, np.newaxis, 1],
                                 turbines_coords[np.newaxis, :, 0], turbines_coords[np.newaxis, :, 1])

# Find the minimum distance for each building
min_distances = distances.min(axis=1)

# Filter buildings where the closest turbine is within 1km
close_buildings_mask = min_distances <= 1  # 1km
filtered_buildings = buildings_coords[close_buildings_mask]

# Result
print(filtered_buildings)

           xtf_id  hubHeight  diameter  ratedPower  longitude   latitude
index                                                                   
0       turbine_5        100        71        2000   7.037331  46.159843
1       turbine_8         61        52         900   8.086715  46.990003
2       turbine_9         46        40         600   8.616332  46.654224
8      turbine_31         78        82        2000   7.100815  47.302711
9      turbine_32         99        82        2000   7.052194  46.126610
10     turbine_33         55        44         900   8.612466  46.654893
11     turbine_34         95        90        2000   7.015705  47.176891
12     turbine_35        108        82        2300   6.955356  47.199788
13     turbine_38         85        71        2310   8.374565  46.463039
14     turbine_39         50        54         950   8.087087  46.987077
15     turbine_41         99       101        3000   7.145424  46.131138
16     turbine_42         55        44         900 

In [1]:
import sqlite3
import pandas as pd
import xarray as xr

# Connexion à la base de données SQLite
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

# Exécuter une requête SQL
cursor.execute("SELECT EGID, GKODE, GKODN, GKAT FROM building WHERE GKAT<1050")
rows = cursor.fetchall()

# Fermer la connexion
conn.close()

# Stocker les résultats dans un tableau (liste)
data = list(rows)
transformer = pyproj.Transformer.from_crs(swiss, wgs84, always_xy=True)

def convert_coords(gkode, gkodn):
    # Check if the coordinates are empty strings
    if gkode == '' or gkodn == '':
        # Handle the empty string case (e.g., return None or a default value)
        return None, None
    else:
        # Convert string coordinates to float
        gkode_float = float(gkode)
        gkodn_float = float(gkodn)
        
        # Perform the transformation
        lon, lat = transformer.transform(gkode_float, gkodn_float)
        return lat, lon

# Convert and store the coordinates
converted_data = []
for row in data:
    egid, gkode, gkodn, gkat = row
    lat, lon = convert_coords(gkode, gkodn)
    
    # Skip rows with missing coordinates
    if lat is not None and lon is not None:
        converted_data.append((egid, lat, lon, gkat))


# Assuming converted_data is structured as [(egid, lat, lon, gkat), ...]

# Convert converted_data to a pandas DataFrame
df = pd.DataFrame(converted_data, columns=['EGID', 'Latitude', 'Longitude', 'GKAT'])

# Convert the DataFrame to an xarray DataArray
houses_array = xr.DataArray(df, dims=["building", "attribute"])

# Print the xarray DataArray
print(houses_array)

NameError: name 'pyproj' is not defined

In [6]:
import numpy as np
import pandas as pd
import pyproj
import sqlite3
import xarray as xr

# Load wind turbine data from a CSV file
df = pd.read_csv('Windraeder.csv', encoding='ISO-8859-1')

# Filter out only operational turbines
df = df[df['operationalStatus'] == 'operationalStatus_normalbetrieb']

# Define coordinate systems for transformation
swiss = pyproj.CRS('EPSG:2056')  # Swiss coordinate system (LV95)
wgs84 = pyproj.CRS('EPSG:4326')  # WGS84 coordinate system
transformer = pyproj.Transformer.from_crs(swiss, wgs84, always_xy=True)

# Function to convert Swiss coordinates to WGS84
def swiss_to_wgs84(x, y):
    lon, lat = transformer.transform(x, y)
    return lon, lat

# Function to handle coordinate conversion with error checking
def convert_coords(gkode, gkodn):
    if gkode == '' or gkodn == '':
        return None, None
    else:
        gkode_float = float(gkode)
        gkodn_float = float(gkodn)
        lon, lat = transformer.transform(gkode_float, gkodn_float)
        return lat, lon

# Convert coordinates of wind turbines
df[['longitude', 'latitude']] = df.apply(lambda row: swiss_to_wgs84(row['x'], row['y']), axis=1, result_type='expand')

# Select relevant columns and convert DataFrame to xarray
selected_columns = ['xtf_id', 'hubHeight', 'diameter', 'ratedPower', 'latitude', 'longitude']
df_selected = df[selected_columns]
xarray_data = df_selected.to_xarray()

# Connect to SQLite database and fetch building data
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT EGID, GKODE, GKODN, GKAT FROM building WHERE GKAT<1050")
rows = cursor.fetchall()
conn.close()

# Process building data and convert coordinates
converted_data = [(row[0], *convert_coords(row[1], row[2]), row[3]) for row in rows if convert_coords(row[1], row[2]) != (None, None)]

# Convert building data to pandas DataFrame and then to xarray
df_buildings = pd.DataFrame(converted_data, columns=['EGID', 'Latitude', 'Longitude', 'GKAT'])
buildings_array = df_buildings.to_xarray()

# Extract coordinates for distance calculation
buildings_coords = np.array([buildings_array['Latitude'].values, buildings_array['Longitude'].values]).T
turbines_coords = np.array([xarray_data['latitude'].values, xarray_data['longitude'].values]).T


# Vectorized haversine function
def haversine_vectorized(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Multiply by Earth's radius to get kilometers
    return km    
# Calculate distances between each building and turbine
distances = haversine_vectorized(buildings_coords[:, np.newaxis, 0], buildings_coords[:, np.newaxis, 1],
                                 turbines_coords[np.newaxis, :, 0], turbines_coords[np.newaxis, :, 1])

# Find the minimum distance for each building and filter those within 1km of a turbine
min_distances = distances.min(axis=1)
close_buildings_mask = min_distances <= 1  # 1km
filtered_buildings = buildings_coords[close_buildings_mask]

# Display buildings close to turbines
print(filtered_buildings)

# Additional analysis: Find and display all turbines within 1 km of each building
results = []
for i, building_coord in enumerate(buildings_coords):
    distances_to_turbines = haversine_vectorized(building_coord[0], building_coord[1], turbines_coords[:, 0], turbines_coords[:, 1])
    turbines_within_1km = np.where(distances_to_turbines <= 1)[0]
    for turbine_index in turbines_within_1km:
        results.append((building_coord[0], building_coord[1], distances_to_turbines[turbine_index]))

numpy_results = np.array(results, dtype=[('Latitude', float), ('Longitude', float), ('Distance to Turbine (km)', float)])
print(numpy_results)


[[47.18403558  7.0354645 ]
 [47.18251044  7.03170183]
 [47.18376991  7.02950558]
 ...
 [47.30610968  7.11514695]
 [47.30457422  7.11244844]
 [47.29542147  7.10015454]]
[(47.18403558, 7.0354645 , 0.87980363)
 (47.18251044, 7.03170183, 0.80488438)
 (47.18251044, 7.03170183, 0.75976025) ...
 (47.30457422, 7.11244844, 0.90135097)
 (47.30457422, 7.11244844, 0.41435749)
 (47.29542147, 7.10015454, 0.8121131 )]


In [7]:
# Count the number of buildings within 1 km of a wind turbine
num_buildings_within_1km = len(filtered_buildings)

# Print summary for the user
print(f"Number of buildings within 1 km of a wind turbine: {num_buildings_within_1km}")

Number of buildings within 1 km of a wind turbine: 796
